In [ ]:
# Set this to your RuleBender workspace if the default isn't right
home = %env HOME
rb_workspace = "{}/RuleBender-workspace/abc_ssa/results/abc_ssa/".format(home)
rb_workspace

In [ ]:
import glob
import pandas as pd
import ipywidgets as widgets
import cufflinks as cf
import plotly.offline as py
import plotly.graph_objs as go
import os
from pprint import pprint

# Set offline to true or you'll have to create a plotly account
cf.set_config_file(offline=True, world_readable=False, theme='pearl')
py.init_notebook_mode() # graphs charts inline (IPython).

In [ ]:
# Grab all the gdat files listed under the RB workspace

glob_str = '{}/**/*.gdat'.format(rb_workspace)
files_to_plot = [fname for fname in glob.iglob(glob_str, recursive=True)]
files_to_plot.sort()

In [ ]:
def gdat_to_df(fname):
    f=open(fname, 'r')
    line1=f.readline()
    column_titles=line1.replace('#','').split()
    basename = os.path.basename(fname)
    basename_noext, _ = os.path.splitext(basename)
    df=pd.read_csv(f, sep='\s+', names=column_titles)
    df = df.set_index('time')
    column_titles = column_titles[1:]
    return df, [(basename_noext, c) for c in column_titles]
    #return df

def plot_df(df, xlog=False, ylog=False):
    data = [go.Scatter(y=df[title], name=title) for title in list(df)]

    layout = go.Layout(
        xaxis=dict(
            type='log' if xlog else 'linear' ,
            title='time (s)',
        ),
        yaxis=dict(
            type='log' if ylog else 'linear',
            title='counts',
        )
    )

    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

In [ ]:
# Create dataframe from single gdat file
df, _ = gdat_to_df(files_to_plot[0])
# A panda dataframe prints in tabular format in a Jupyter notebook
df.head()

In [ ]:
# default matplotlib plot of panda's dataframe
df.plot()

In [ ]:
# Create large dataframe from multiple gdat files
# Set up multiindex column to organize output
# Top level is file name, which highlights the parameter in question
# Bottom level species names (A, B, C, etc)

files_to_plot.sort()
df_list = []
column_multi_list = []
for fname in files_to_plot:
    df, column_multi = gdat_to_df(fname)
    column_multi_list.extend(column_multi)
    df_list.append(df)
big_df = pd.concat(df_list, axis=1)
big_df.columns = pd.MultiIndex.from_tuples(
    column_multi_list,
    names=['file', 'species'])
big_df.tail()

In [ ]:
# There are many ways to slice and manipulate a Panda's dataframe.
# See here for more info: https://pandas.pydata.org/pandas-docs/stable/10min.html

all_b = big_df.loc[:, (slice(None), 'B')]

In [ ]:
# basic Scatter plotly plot
# data = [go.Scatter(y=df[title], name=title) for title in list(df)]
# py.iplot(data)
all_b.iplot()